In [66]:
#libraries

import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.dates as mandates
from sklearn import linear_model
from keras.models import Sequential
from keras.layers import Dense

import keras.backend as K
from keras.callbacks import EarlyStopping
#from Keras.models import load_model
from keras.layers import LSTM
from keras.utils.vis_utils import plot_model

In [67]:
#Get the Dataset
df1=pd.read_csv('../input/rocketcryptochallenge/train_dataset.csv')
df1.head()

In [68]:
print("shape: ",df1.shape)
print('null: ',df1.isnull().values.any())

In [69]:
df1.index_col='Date'
df=df1[df1['symbol']=='BTC']
df
    

In [70]:
df.parse_dates=True
df.infer_datetime_format=True

In [71]:
#plotting the actual price with time
df['target'].plot()

In [72]:
#Setting the Variables
output_var = pd.DataFrame(df['target'])

features = ['feature_58', 'feature_440','feature_692','feature_694','feature_696','feature_700','feature_710','feature_720','feature_1017','feature_1389','feature_1398','feature_1399','feature_1400','feature_1406','feature_1408','feature_1410','feature_1466','feature_1742','feature_1751','feature_1753','feature_1778','feature_1780']

In [73]:
#Scaling

# scaler = MinMaxScaler()
# feature_transform = scaler.fit_transform(df[features])
# feature_transform= pd.DataFrame(columns=features, data=feature_transform, index=df.index)
# feature_transform.head()

In [74]:
#Splitting to Training set and Test set
timesplit= TimeSeriesSplit(n_splits=10)
for train_index, test_index in timesplit.split(feature_transform):
        X_train, X_test = feature_transform[:len(train_index)], feature_transform[len(train_index): (len(train_index)+len(test_index))]
        y_train, y_test = output_var[:len(train_index)].values.ravel(), output_var[len(train_index): (len(train_index)+len(test_index))].values.ravel()

In [75]:
#Process the data for LSTM
trainX =np.array(X_train)
testX =np.array(X_test)
X_train = trainX.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = testX.reshape(X_test.shape[0], 1, X_test.shape[1])

In [76]:
import tensorflow as tf

In [77]:
import keras

In [88]:
#Building the LSTM Model
lstm = Sequential()
lstm.add(LSTM(64, input_shape=(1, trainX.shape[1]), activation='relu', return_sequences=False))
#lstm.add(Dense(32))
lstm.add(Dense(1))
lstm.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005))
#plot_model(lstm, show_shapes=True, show_layer_names=True)
lstm.summary()

In [92]:
#Model Training
history=lstm.fit(X_train, y_train, epochs=500, batch_size=4, verbose=1, shuffle=False)

In [93]:
#LSTM Prediction
y_pred= lstm.predict(X_test)

In [94]:
#Predicted vs True Adj Close Value – LSTM
plt.plot(y_test, label='Actual price')
plt.plot(y_pred, label='Predicted value')
plt.xlim([0,20])
plt.title('Prediction by LSTM')
plt.xlabel('Time Scale')
plt.ylabel('Scaled USD')
plt.legend()
plt.show()